## Encoding dataset  :  Credits

In [288]:
import pandas as pd
import numpy as np
#Encoding modules
import ast
import sys
sys.path.append('../modules')
from encoding_module import json_to_DF
from encoding_module import onehot_json
import json
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
#Plots
from plot_module import barPlot
from plot_module import featureAnalysis
#API
import requests
import concurrent.futures
#NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#Ignore Warning
import warnings
warnings.filterwarnings("ignore")

In [3]:
credits_raw =  pd.read_csv("../../dataset/credits_raw.csv")

In [4]:
credits_raw.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [5]:
credits_raw.shape

(45476, 3)

In [7]:
movies_metadata = pd.read_csv('encoded/movies_metadata_encoded.csv')

In [8]:
ID = movies_metadata['id'].tolist()

In [10]:
credits_raw = credits_raw[credits_raw['id'].isin(ID)]

In [11]:
credits_raw

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45468,"[{'cast_id': 2, 'character': 'Branwall', 'cred...","[{'credit_id': '5403d669c3a3682d9800427d', 'de...",289923
45469,"[{'cast_id': 1, 'character': 'Kira (as Cassand...","[{'credit_id': '5757f36ac3a3687d6f000e8a', 'de...",222848
45470,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",30840
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109


In [12]:
credits_raw=credits_raw.drop_duplicates()
credits_raw.shape

(36389, 3)

We want to pick the 500 top directors, because a lot of movies don't have the other roles like writers or producers etc.

In [50]:
credits_raw.loc[0]['crew']

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [107]:
def extract_directors(dataset):

    crew_df = pd.DataFrame(columns=['id', 'directors'])
    
    for index, movie in dataset.iterrows():
        movie_id = movie['id']
        crew = movie['crew']
        directors = [member['name'] for member in crew if member['job'] == 'Director']

        crew_df = crew_df.append({'id': movie_id,'directors': directors}, ignore_index=True)

    return crew_df

In [111]:
crew_df = extract_directors(credits_raw)
crew_df

,id,directors
0,862,[John Lasseter]
1,8844,[Joe Johnston]
2,15602,[Howard Deutch]
3,31357,[Forest Whitaker]
4,11862,[Charles Shyer]
...,...,...
36384,289923,[Ben Rock]
36385,222848,[Aaron Osborne]
36386,30840,[John Irvin]
36387,111109,[Lav Diaz]


In [129]:
directors = [director for sublist in crew_df['directors'] for director in sublist]
director_counts = Counter(directors)
top_directors = [director for director, count in director_counts.most_common(500)]
print(top_directors)

['Michael Curtiz', 'John Ford', 'Woody Allen', 'Werner Herzog', 'Alfred Hitchcock', 'Sidney Lumet', 'Jean-Luc Godard', 'Henry Hathaway', 'Raoul Walsh', 'George Cukor', 'John Huston', 'William A. Wellman', 'Ingmar Bergman', 'Richard Thorpe', 'Fritz Lang', 'Mervyn LeRoy', 'Martin Scorsese', 'Clint Eastwood', 'Robert Wise', 'Roger Corman', 'Claude Chabrol', 'Howard Hawks', 'Anthony Mann', 'J. Lee Thompson', 'Norman Taurog', 'Takashi Miike', 'Spike Lee', 'W.S. Van Dyke', 'Richard Fleischer', 'Gordon Douglas', 'Rainer Werner Fassbinder', 'Roy Del Ruth', 'Robert Altman', 'Steven Spielberg', 'Vincente Minnelli', 'Henry Koster', 'Wim Wenders', 'John Frankenheimer', 'Blake Edwards', 'Akira Kurosawa', 'William Beaudine', 'Lloyd Bacon', 'Michael Apted', 'William Wyler', 'Robert Stevenson', 'Edward Dmytryk', 'John Sturges', 'Robert Aldrich', 'Buster Keaton', 'Lucio Fulci', 'Anatole Litvak', 'Clarence Brown', 'Johnnie To', 'Georges Méliès', 'Stephen Frears', 'Steven Soderbergh', 'Robert Z. Leonard'

In [132]:
directors_df = pd.DataFrame(columns=top_directors)

In [133]:
directors_df

,Michael Curtiz,John Ford,Woody Allen,Werner Herzog,Alfred Hitchcock,Sidney Lumet,Jean-Luc Godard,Henry Hathaway,Raoul Walsh,George Cukor,...,Stephen Hopkins,Richard Attenborough,Roger Vadim,Randal Kleiser,Tsai Ming-liang,Gene Kelly,Mark Sandrich,Jack Kinney,Andrei Tarkovsky,Claire Denis


In [134]:
for index, row in crew_df.iterrows():
    for director in row['directors']:
        if director in top_directors:
            directors_df.loc[index, director] = 1

In [136]:
directors_df = directors_df.fillna(0)

In [138]:
directors_df["Alfred Hitchcock"].value_counts()

0    8898
1      48
Name: Alfred Hitchcock, dtype: int64

In [246]:
merged_df_direct = crew_df.merge(directors_df, how='left', left_index=True, right_index=True).fillna(0)

In [249]:
merged_df_direct = merged_df_direct.drop(columns = "directors")

In [250]:
merged_df_direct

,id,Michael Curtiz,John Ford,Woody Allen,Werner Herzog,Alfred Hitchcock,Sidney Lumet,Jean-Luc Godard,Henry Hathaway,Raoul Walsh,...,Stephen Hopkins,Richard Attenborough,Roger Vadim,Randal Kleiser,Tsai Ming-liang,Gene Kelly,Mark Sandrich,Jack Kinney,Andrei Tarkovsky,Claire Denis
0,862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36384,289923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36385,222848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36386,30840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36387,111109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [251]:
merged_df_direct.to_csv('directors_df.csv', index=False)

#### Extract Actors

In [185]:
ast.literal_eval(credits_raw.loc[0]['cast'])

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [197]:
def extract_actors(dataset):

    cast_df = pd.DataFrame(columns=['id', 'actors','main_cast_gender','women_actors_count','men_actors_count'])

    ids_have_to_drop = []
    
    for index, movie in dataset.iterrows():
        movie_id = movie['id']
        cast = ast.literal_eval(movie["cast"])
        if len(cast) == 0:
            ids_have_to_drop.append(movie_id)
        else: 
            gender = cast[0]["gender"]
            actors = [member['name'] for member in cast]

            women_count = 0
            men_count = 0

            for member in cast:
                if member['gender'] == 1:
                    women_count += 1
                elif member['gender'] == 2:
                    men_count += 1

            cast_df = cast_df.append({'id': movie_id,'actors': actors,'main_cast_gender':gender,'women_actors_count':women_count,'men_actors_count':men_count}, ignore_index=True)

    return cast_df

In [198]:
cast_df = extract_actors(credits_raw)
cast_df

,id,actors,main_cast_gender,women_actors_count,men_actors_count
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",2,3,9
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",2,6,9
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",2,3,4
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...",1,4,6
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",2,5,6
...,...,...,...,...,...
35626,289923,"[Monty Bane, Lucy Butler, David Grammer, Bill ...",2,2,3
35627,222848,"[Lisa Boyle, Kena Land, Zaneta Polard, Don Yan...",1,1,3
35628,30840,"[Patrick Bergin, Uma Thurman, David Morrissey,...",2,1,3
35629,111109,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",1,1,1


In [297]:
cast_df[cast_df["main_cast_gender"] == 0]

,id,actors,main_cast_gender,women_actors_count,men_actors_count
36,139405,"[Peter Reznick, John McDonough, Avi Hoffman, M...",0,0,1
38,9603,"[Alicia Silverstone, Stacey Dash, Brittany Mur...",0,6,6
39,34615,"[James Earl Jones, Charles S. Dutton, Jack Rob...",0,1,2
48,8391,"[Pascale Bussières, Rachael Crawford, Henry Cz...",0,1,2
64,40628,"[Jerry Barone, Al Pacino, Mary Elizabeth Mastr...",0,1,3
...,...,...,...,...,...
35618,44333,[Georges Méliès],0,0,0
35619,49277,[Georges Méliès],0,0,0
35621,49280,[Georges Méliès],0,0,0
35622,106807,"[Jeanne d'Alcy, Georges Méliès]",0,0,0


In [301]:
cast_df[cast_df["main_cast_gender"] == 1]

,id,actors,main_cast_gender,women_actors_count,men_actors_count
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...",1,4,6
14,1408,"[Geena Davis, Matthew Modine, Frank Langella, ...",1,2,20
16,4584,"[Kate Winslet, Emma Thompson, Hugh Grant, Tom ...",1,7,9
21,1710,"[Sigourney Weaver, Holly Hunter, Will Patton, ...",1,3,7
23,12665,"[Mary Steenburgen, Sean Patrick Flanery, Lance...",1,1,4
...,...,...,...,...,...
35617,14008,"[Hilary Duff, Christy Carlson Romano, Gary Col...",1,5,5
35623,404604,"[Sridevi Kapoor, Sajal Ali, Akshaye Khanna, Na...",1,2,3
35627,222848,"[Lisa Boyle, Kena Land, Zaneta Polard, Don Yan...",1,1,3
35629,111109,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",1,1,1


In [302]:
cast_df[cast_df["main_cast_gender"] == 2]

,id,actors,main_cast_gender,women_actors_count,men_actors_count
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",2,3,9
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",2,6,9
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",2,3,4
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",2,5,6
5,949,"[Al Pacino, Robert De Niro, Val Kilmer, Jon Vo...",2,15,30
...,...,...,...,...,...
35613,63266,"[Pyotr Fyodorov, Petr Tomashevsky, Agniya Kuzn...",2,1,1
35620,14885,"[Jimmy Bennett, Peter Cullen, Jim Cummings, Jo...",2,1,10
35625,84419,"[Rondo Hatton, Robert Lowery, Virginia Grey, B...",2,3,7
35626,289923,"[Monty Bane, Lucy Butler, David Grammer, Bill ...",2,2,3


In [257]:
empty_cast_id = []
for index, row in credits_raw.iterrows():
    cast = ast.literal_eval(row['cast'])
    if len(cast) == 0:
        empty_cast_id.append(row["id"])

In [259]:
len(empty_cast_id)

758

In [220]:
actors = [actor for sublist in cast_df['actors'] for actor in sublist]
actor_counts = Counter(actors)
top_actors = [actor for actor, count in actor_counts.most_common(1000)]

In [217]:
# columns = top_actors
# columns.insert(0, 'main_cast_gender') 
# columns.insert(0, 'women_actors_count')
# columns.insert(0, 'men_actors_count')

In [221]:
actors_df = pd.DataFrame(columns=top_actors)

In [222]:
actors_df

,Bess Flowers,Christopher Lee,Samuel L. Jackson,John Wayne,Jackie Chan,John Carradine,Gérard Depardieu,Michael Caine,Donald Sutherland,Robert De Niro,...,Natasha Lyonne,Robert Carradine,Olivier Gourmet,Francis Ford,Leoda Richards,John Le Mesurier,Ken Lo,John Michael Higgins,Aaron Eckhart,Andy Serkis


In [223]:
for index, row in cast_df.iterrows():
    for actor in row['actors']:
        if actor in top_actors:
            actors_df.loc[index, actor] = 1

In [224]:
actors_df = actors_df.fillna(0)

In [225]:
actors_df

,Bess Flowers,Christopher Lee,Samuel L. Jackson,John Wayne,Jackie Chan,John Carradine,Gérard Depardieu,Michael Caine,Donald Sutherland,Robert De Niro,...,Natasha Lyonne,Robert Carradine,Olivier Gourmet,Francis Ford,Leoda Richards,John Le Mesurier,Ken Lo,John Michael Higgins,Aaron Eckhart,Andy Serkis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35625,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35628,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
merged_df = cast_df.merge(actors_df, how='left', left_index=True, right_index=True).fillna(0)

In [243]:
merged_df = merged_df.drop(columns = "actors")

In [245]:
merged_df

,id,main_cast_gender,women_actors_count,men_actors_count,Bess Flowers,Christopher Lee,Samuel L. Jackson,John Wayne,Jackie Chan,John Carradine,...,Natasha Lyonne,Robert Carradine,Olivier Gourmet,Francis Ford,Leoda Richards,John Le Mesurier,Ken Lo,John Michael Higgins,Aaron Eckhart,Andy Serkis
0,862,2,3,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8844,2,6,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15602,2,3,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,1,4,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11862,2,5,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35626,289923,2,2,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35627,222848,1,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35628,30840,2,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35629,111109,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [244]:
merged_df.to_csv('actors_df.csv', index=False)

In [281]:
merged_df_credits = merged_df.merge(merged_df_direct, on='id', how='inner').fillna(0)

In [282]:
merged_df_credits

,id,main_cast_gender,women_actors_count,men_actors_count,Bess Flowers,Christopher Lee,Samuel L. Jackson,John Wayne,Jackie Chan_x,John Carradine,...,Stephen Hopkins,Richard Attenborough,Roger Vadim,Randal Kleiser,Tsai Ming-liang,Gene Kelly,Mark Sandrich,Jack Kinney,Andrei Tarkovsky,Claire Denis
0,862,2,3,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8844,2,6,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15602,2,3,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,1,4,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11862,2,5,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35638,289923,2,2,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35639,222848,1,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35640,30840,2,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35641,111109,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [323]:
movies_metadata = pd.read_csv("./encoded/movies_metadata_encoded.csv")

In [324]:
movies_metadata

,id,imdb_id,popularity,release_date,runtime,vote_average,vote_count,adult_False,adult_True,Single,...,Latviešu,isiZulu,ਪੰਜਾਬੀ,euskera,Bahasa melayu,Wolof,others_spoken_languages,year,Runtime Category,Vote
0,862,tt0114709,21.946943,1995-10-30,81.0,7.7,5415.0,1,0,0,...,0,0,0,0,0,0,1,1995,2,41695.5
1,8844,tt0113497,17.015539,1995-12-15,104.0,6.9,2413.0,1,0,1,...,0,0,0,0,0,0,1,1995,2,16649.7
2,15602,tt0113228,11.712900,1995-12-22,101.0,6.5,92.0,1,0,0,...,0,0,0,0,0,0,1,1995,2,598.0
3,31357,tt0114885,3.859495,1995-12-22,127.0,6.1,34.0,1,0,1,...,0,0,0,0,0,0,1,1995,2,207.4
4,11862,tt0113041,8.387519,1995-02-10,106.0,5.7,173.0,1,0,0,...,0,0,0,0,0,0,1,1995,2,986.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36378,289923,tt0252966,0.386450,2000-10-03,30.0,7.0,1.0,1,0,1,...,0,0,0,0,0,0,1,2000,1,7.0
36379,222848,tt0112613,0.661558,1995-01-01,85.0,3.5,1.0,1,0,1,...,0,0,0,0,0,0,1,1995,2,3.5
36380,30840,tt0102797,5.683753,1991-05-13,104.0,5.7,26.0,1,0,1,...,0,0,0,0,0,0,1,1991,2,148.2
36381,111109,tt2028550,0.178241,2011-11-17,360.0,9.0,3.0,1,0,1,...,0,0,0,0,0,0,1,2011,4,27.0


In [331]:
total_merged = movies_metadata.merge(merged_df_credits, on='id', how='inner').fillna(0)

In [332]:
total_merged

,id,imdb_id,popularity,release_date,runtime,vote_average,vote_count,adult_False,adult_True,Single,...,Stephen Hopkins,Richard Attenborough,Roger Vadim,Randal Kleiser,Tsai Ming-liang,Gene Kelly,Mark Sandrich,Jack Kinney,Andrei Tarkovsky,Claire Denis
0,862,tt0114709,21.946943,1995-10-30,81.0,7.7,5415.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8844,tt0113497,17.015539,1995-12-15,104.0,6.9,2413.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15602,tt0113228,11.712900,1995-12-22,101.0,6.5,92.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,tt0114885,3.859495,1995-12-22,127.0,6.1,34.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11862,tt0113041,8.387519,1995-02-10,106.0,5.7,173.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35638,289923,tt0252966,0.386450,2000-10-03,30.0,7.0,1.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35639,222848,tt0112613,0.661558,1995-01-01,85.0,3.5,1.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35640,30840,tt0102797,5.683753,1991-05-13,104.0,5.7,26.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35641,111109,tt2028550,0.178241,2011-11-17,360.0,9.0,3.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [333]:
total_merged_unique = total_merged.drop_duplicates(subset='id', keep='first')

In [334]:
total_merged_unique

,id,imdb_id,popularity,release_date,runtime,vote_average,vote_count,adult_False,adult_True,Single,...,Stephen Hopkins,Richard Attenborough,Roger Vadim,Randal Kleiser,Tsai Ming-liang,Gene Kelly,Mark Sandrich,Jack Kinney,Andrei Tarkovsky,Claire Denis
0,862,tt0114709,21.946943,1995-10-30,81.0,7.7,5415.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8844,tt0113497,17.015539,1995-12-15,104.0,6.9,2413.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15602,tt0113228,11.712900,1995-12-22,101.0,6.5,92.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31357,tt0114885,3.859495,1995-12-22,127.0,6.1,34.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11862,tt0113041,8.387519,1995-02-10,106.0,5.7,173.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35638,289923,tt0252966,0.386450,2000-10-03,30.0,7.0,1.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35639,222848,tt0112613,0.661558,1995-01-01,85.0,3.5,1.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35640,30840,tt0102797,5.683753,1991-05-13,104.0,5.7,26.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35641,111109,tt2028550,0.178241,2011-11-17,360.0,9.0,3.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Encode the gender of the main actor

In [336]:
gender = total_merged_unique["main_cast_gender"].values.reshape(-1, 1)
encoder = OneHotEncoder()
onehot_encoded = encoder.fit_transform(gender)

In [337]:
onehot_df = pd.DataFrame(onehot_encoded.toarray(), columns=["main_cast_unknown","main_cast_woman", "main_cast_man"])

In [341]:
data_encoded = pd.concat([total_merged_unique.reset_index(drop=True), onehot_df], axis=1)

In [344]:
data_encoded = data_encoded.drop(columns = ["main_cast_gender"])

In [346]:
data_encoded = data_encoded.drop(columns = ["main_cast_unknown"])

In [347]:
data_encoded

,id,imdb_id,popularity,release_date,runtime,vote_average,vote_count,adult_False,adult_True,Single,...,Roger Vadim,Randal Kleiser,Tsai Ming-liang,Gene Kelly,Mark Sandrich,Jack Kinney,Andrei Tarkovsky,Claire Denis,main_cast_woman,main_cast_man
0,862,tt0114709,21.946943,1995-10-30,81.0,7.7,5415.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8844,tt0113497,17.015539,1995-12-15,104.0,6.9,2413.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,15602,tt0113228,11.712900,1995-12-22,101.0,6.5,92.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,31357,tt0114885,3.859495,1995-12-22,127.0,6.1,34.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,11862,tt0113041,8.387519,1995-02-10,106.0,5.7,173.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35620,289923,tt0252966,0.386450,2000-10-03,30.0,7.0,1.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35621,222848,tt0112613,0.661558,1995-01-01,85.0,3.5,1.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
35622,30840,tt0102797,5.683753,1991-05-13,104.0,5.7,26.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35623,111109,tt2028550,0.178241,2011-11-17,360.0,9.0,3.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [348]:
data_encoded.to_csv('movies_metadata3_encoded.csv', index=False)